In [2]:
import pandas as pd # Used to open CSV files 
import numpy as np # Used for matrix operations
#import cv2 # Used for image augmentation
from matplotlib import pyplot as plt
np.random.seed(666)

In [1]:
import tensorflow as tf

c:\users\pankaz das\appdata\local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\pankaz das\appdata\local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\pankaz das\appdata\local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [3]:
tf.__version__

'1.9.0'

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


AttributeError: module 'tensorflow.python.keras.backend' has no attribute 'get_graph'

In [6]:
df_train = pd.read_json("./input/train.json/train.json") # this is a dataframe

In [39]:
# scaling of the images
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
#        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
#        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
#        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        # Rescale 2
        a = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.min()) / (band_3.max() - band_3.min())
        
        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [45]:
# traing data with scaled images
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

In [41]:
# incident angle
df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

In [42]:
# taking those image data where incident angle is not zero
Xtrain = Xtrain[idx_tr[0],...]
Ytrain = Ytrain[idx_tr[0]]

In [11]:
# get horizontal and vertical version of the same images
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [12]:
#traing data with original, horizontal and vertical version of the image
Xtr_more = get_more_images(Xtrain) 
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

In [43]:
# define layers with keras of the model

def getModel():
    #Build keras model
    
    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

model = getModel()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [44]:
# batch size and callbacks parameter during traing procedures for early stopping and changing learning rate
batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=0, epsilon=1e-4, mode='min')

In [46]:
# Let's view progress 
#history = model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)
history = model.fit(Xtrain, Ytrain, batch_size=batch_size, epochs=25, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)

Train on 1203 samples, validate on 401 samples
Epoch 1/25
1203/1203 [==============================] - 22s - loss: 0.6947 - acc: 0.5104 - val_loss: 0.7257 - val_acc: 0.3441
Epoch 2/25
1203/1203 [==============================] - 18s - loss: 0.6915 - acc: 0.5137 - val_loss: 0.6921 - val_acc: 0.5711
Epoch 3/25
1203/1203 [==============================] - 18s - loss: 0.6936 - acc: 0.4996 - val_loss: 0.6840 - val_acc: 0.6284
Epoch 4/25
1203/1203 [==============================] - 18s - loss: 0.6664 - acc: 0.6176 - val_loss: 0.6346 - val_acc: 0.6783
Epoch 5/25
1203/1203 [==============================] - 19s - loss: 0.6389 - acc: 0.6259 - val_loss: 0.6181 - val_acc: 0.6010
Epoch 6/25
1203/1203 [==============================] - 18s - loss: 0.5687 - acc: 0.7149 - val_loss: 0.6440 - val_acc: 0.5786
Epoch 7/25
1203/1203 [==============================] - 18s - loss: 0.5668 - acc: 0.7174 - val_loss: 0.5192 - val_acc: 0.6883
Epoch 8/25
1203/1203 [==============================] - 18s - loss: 0.4

In [24]:
print(history.history.keys())
#
fig1 = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#
fig1.savefig('accuracy.png')
#---------------------------------------------------------------------------------------

dict_keys(['val_loss', 'val_acc', 'loss', 'acc', 'lr'])


In [25]:
fig2 = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower left')
#
fig2.savefig('loss.png')

In [18]:
model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1471/1471 [==============================] - 5s     
Train score: 0.11366522358
Train accuracy: 0.953772943576


In [19]:
# load test data
df_test = pd.read_json("./input/test.json/data/processed/test.json")
df_test.inc_angle = df_test.inc_angle.replace('na',0)

In [26]:
Xtest = get_scaled_imgs(df_test)

In [28]:
Xte_more = get_more_images(Xtest)

MemoryError: 

In [27]:
pred_test = model.predict(Xtest)

In [21]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

         id    is_iceberg
0  5941774d  2.311683e-02
1  4023181e  8.810326e-01
2  b20200e4  1.024917e-01
3  e7f018bb  9.998856e-01
4  4371c8c3  4.436552e-01
5  a8d9b1fd  1.950237e-01
6  29e7727e  3.502675e-03
7  92a51ffb  9.998951e-01
8  c769ac97  1.185725e-08
9  aee0547d  5.381195e-08


In [22]:
submission.to_csv('submission_final.csv', index=False)